In [5]:
from datetime import datetime, timedelta
from collections import defaultdict

# Provided item list from the PDF
item_list = [
    {"idx": 1, "item_code": "Executive Desk (4*2)", "sales_description": "Dedicated Executive Desk", "qty": 10, "rate": "1000", "amount": "10000", "start_date": "2023-11-01", "stop_date": "2024-10-17"},
    {"idx": 2, "item_code": "Executive Desk (4*2)", "qty": "10", "rate": "1080", "amount": "10800", "start_date": "2024-10-18", "stop_date": "2025-10-31"},
    {"idx": 3, "item_code": "Executive Desk (4*2)", "qty": 15, "rate": "1080", "amount": "16200", "start_date": "2024-11-01", "stop_date": "2025-10-31"},
    {"idx": 4, "item_code": "Executive Desk (4*2)", "qty": 5, "rate": "1000", "amount": "5000", "start_date": "2024-11-01", "stop_date": "2025-10-31"},
    {"idx": 5, "item_code": "Manager Cabin", "qty": 5, "rate": 5000, "amount": 25000, "start_date": "2024-11-01", "stop_date": "2025-10-31"},
    {"idx": 6, "item_code": "Manager Cabin", "qty": 7, "rate": "5000", "amount": 35000, "start_date": "2024-12-15", "stop_date": "2025-10-31"},
    {"idx": 7, "item_code": "Manager Cabin", "qty": 10, "rate": 4600, "amount": 46000, "start_date": "2023-11-01", "stop_date": "2024-10-17"},
    {"idx": 8, "item_code": "Parking (2S)", "qty": 10, "rate": 1000, "amount": 10000, "start_date": "2024-11-01", "stop_date": "2025-10-31"},
    {"idx": 9, "item_code": "Parking (2S)", "qty": 10, "rate": 0, "amount": 0, "start_date": "2024-11-01", "stop_date": "2025-10-31"},
    {"idx": 10, "item_code": "Executive Desk (4*2)", "qty": "8", "rate": "1100", "amount": "8800", "start_date": "2024-11-15", "stop_date": "2025-01-31"},
    {"idx": 11, "item_code": "Manager Cabin", "qty": "3", "rate": "5200", "amount": "15600", "start_date": "2024-10-10", "stop_date": "2024-11-10"},
    {"idx": 12, "item_code": "Conference Table", "qty": 1, "rate": "20000", "amount": "20000", "start_date": "2024-11-05", "stop_date": "2024-11-20"},
    {"idx": 13, "item_code": "Parking (2S)", "qty": 5, "rate": "1000", "amount": "5000", "start_date": "2024-11-15", "stop_date": "2025-02-28"},
    {"idx": 14, "item_code": "Reception Desk", "qty": 2, "rate": "7000", "amount": "14000", "start_date": "2024-11-01", "stop_date": "2025-03-31"},
    {"idx": 15, "item_code": "Reception Desk", "qty": 1, "rate": "7000", "amount": "7000", "start_date": "2024-11-10", "stop_date": "2024-11-25"},
    {"idx": 16, "item_code": "Breakout Area", "qty": 3, "rate": "3000", "amount": "9000", "start_date": "2024-01-01", "stop_date": "2024-01-31"},
]

# Function to generate the bill
def generate_monthly_bill(item_list: list, target_month: str) -> dict:
    def parse_date(date_str):
        return datetime.strptime(date_str, "%Y-%m-%d")
    
    def get_month_range(month_str):
        start = datetime.strptime(month_str, "%Y-%m")
        if start.month == 12:
            end = start.replace(year=start.year + 1, month=1)
        else:
            end = start.replace(month=start.month + 1)
        return start, end - timedelta(days=1)

    def overlap_days(start1, end1, start2, end2):
        latest_start = max(start1, start2)
        earliest_end = min(end1, end2)
        delta = (earliest_end - latest_start).days + 1
        return max(0, delta)

    month_start, month_end = get_month_range(target_month)
    grouped_items = defaultdict(lambda: {"qty": 0, "amount": 0.0})

    for item in item_list:
        start_date = parse_date(item["start_date"])
        stop_date = parse_date(item["stop_date"])
        
        if stop_date < month_start or start_date > month_end:
            continue
        
        overlap = overlap_days(start_date, stop_date, month_start, month_end)
        if overlap == 0:
            continue

        full_days = (month_end - month_start).days + 1
        rate = float(item["rate"])
        qty = int(item["qty"])

        daily_rate = rate / full_days
        bill_amount = round(daily_rate * overlap * qty, 2)

        billing_period = f"{month_start.strftime('%Y-%m-%d')} to {month_end.strftime('%Y-%m-%d')}"
        group_key = (item["item_code"], rate, billing_period)

        grouped_items[group_key]["qty"] += qty
        grouped_items[group_key]["amount"] += bill_amount

    line_items = []
    total_revenue = 0.0

    for (item_code, rate, billing_period), group in grouped_items.items():
        amount = round(group["amount"], 2)
        total_revenue += amount
        line_items.append({
            "item_code": item_code,
            "rate": rate,
            "qty": group["qty"],
            "amount": amount,
            "billing_period": billing_period
        })

    return {
        "line_items": line_items,
        "total_revenue": round(total_revenue, 2)
    }

# Generation of bill for November 2024
generate_monthly_bill(item_list, "2024-11")



{'line_items': [{'item_code': 'Executive Desk (4*2)',
   'rate': 1080.0,
   'qty': 25,
   'amount': 27000.0,
   'billing_period': '2024-11-01 to 2024-11-30'},
  {'item_code': 'Executive Desk (4*2)',
   'rate': 1000.0,
   'qty': 5,
   'amount': 5000.0,
   'billing_period': '2024-11-01 to 2024-11-30'},
  {'item_code': 'Manager Cabin',
   'rate': 5000.0,
   'qty': 5,
   'amount': 25000.0,
   'billing_period': '2024-11-01 to 2024-11-30'},
  {'item_code': 'Parking (2S)',
   'rate': 1000.0,
   'qty': 15,
   'amount': 12666.67,
   'billing_period': '2024-11-01 to 2024-11-30'},
  {'item_code': 'Parking (2S)',
   'rate': 0.0,
   'qty': 10,
   'amount': 0.0,
   'billing_period': '2024-11-01 to 2024-11-30'},
  {'item_code': 'Executive Desk (4*2)',
   'rate': 1100.0,
   'qty': 8,
   'amount': 4693.33,
   'billing_period': '2024-11-01 to 2024-11-30'},
  {'item_code': 'Manager Cabin',
   'rate': 5200.0,
   'qty': 3,
   'amount': 5200.0,
   'billing_period': '2024-11-01 to 2024-11-30'},
  {'item_code